<a href="https://colab.research.google.com/github/lguloy/House-Pricing-Predictions/blob/work_vijitha/Second_try_Feature_importance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://finalprojectbootcamp.s3.us-east-2.amazonaws.com/all_dumimies_converted_df.csv

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
# Load in train.csv from S3 into a DataFrame
url = "https://finalprojectbootcamp.s3.us-east-2.amazonaws.com/all_dumimies_converted_df.csv"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("all_dumimies_converted_df.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
df.show(10)

#Convert to Pandas to allow train_test_split usage
df = df.select("*").toPandas()
df = df.dropna(how="any")

In [ ]:
#Get the Feature names
import pandas as pd

features = [col for col in df.columns]
print(features)

print(type(features))

In [ ]:
#Import sklearn tools and dependencies
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# Create the feature data and the target

#Feature DataFrame is X
X = df.drop(columns=["Id", "SalePrice"])


#Target Dataframe is y
y = df["SalePrice"]


#####################Create train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)



In [ ]:
# Scale
X_train_scaler = StandardScaler().fit(X_train)
X_test_scaler = StandardScaler().fit(X_test)
X_train_scaled = X_train_scaler.transform(X_train)
X_test_scaled = X_test_scaler.transform(X_test)

In [ ]:
rf = RandomForestClassifier(n_estimators=200)
rf.fit(X_train_scaled, y_train)
print(f' Training score: {rf.score(X_train_scaled, y_train)}')
print(f' Test score: {rf.score(X_test_scaled, y_test)}')

In [ ]:
importances = rf.feature_importances_
features.remove("Id")
features.remove("SalePrice")

sorted(zip(rf.feature_importances_, features), reverse=True)

In [ ]:


features.remove("Id")
features.remove("SalePrice")

sorted(zip(rf.feature_importances_, features), reverse=True)



In [ ]:
[(0.03067184507694324, '1stFlrSF'),
 (0.030661811314982232, 'GrLivArea'),
 (0.030157742992857357, 'LotArea'),
 (0.030058219806715736, 'GarageArea'),
 (0.02988531453794777, 'BsmtUnfSF'),
 (0.028733422469003674, 'TotalBsmtSF'),
 (0.026627014082029762, 'LotFrontage'),
 (0.025583719381550338, 'GarageYrBlt'),
 (0.02533056678489419, 'MoSold'),
 (0.025239368282836506, 'YearBuilt'),